In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the data from numpy files
X_train = np.load('x_train.npy') / 255.0
y_train = np.load('y_train.npy')
X_test = np.load('x_test.npy') / 255.0
y_test = np.load('y_test.npy')

X_train = X_train.reshape(X_train.shape[0], -1)  # Flatten the images


# Display some examples of the input data
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.flatten()
idx = np.random.randint(0, X_train.shape[0], size=10)
for i in range(10):
    axes[i].imshow(X_train[idx[i], :].reshape(28, 28), cmap='gray')
    axes[i].axis('off')  # hide the axes ticks
    axes[i].set_title(str(int(y_train[idx[i]])), color='black', fontsize=25)
plt.show()

# ReLU activation function
def relu(x):
    x[x < 0] = 0
    return x

# Hypothesis function: simple FNN with 1 hidden layer
def h(X, W, b):
    a1 = X
    z1 = np.matmul(X, W[0]) + b[0]
    a2 = relu(z1)
    z2 = np.matmul(a2, W[1])
    s = np.exp(z2)
    total = np.sum(s, axis=1).reshape(-1, 1)
    sigma = s / total
    return sigma

# Loss function: cross entropy with an L^2 regularization
def loss(y_pred, y_true):
    K = 10
    N = len(y_true)
    y_true_one_hot_vec = (y_true[:, np.newaxis] == np.arange(K))
    loss_sample = (np.log(y_pred) * y_true_one_hot_vec).sum(axis=1)
    return -np.mean(loss_sample)

# Backpropagation function
def backprop(W, b, X, y, alpha=1e-4):
    K = 10
    N = X.shape[0]
    a1 = X
    z1 = np.matmul(X, W[0]) + b[0]
    a2 = relu(z1)
    z2 = np.matmul(a2, W[1])
    s = np.exp(z2)
    total = np.sum(s, axis=1).reshape(-1, 1)
    sigma = s / total
    y_one_hot_vec = (y[:, np.newaxis] == np.arange(K))
    delta2 = (sigma - y_one_hot_vec)
    grad_W1 = np.matmul(a2.T, delta2)
    delta1 = np.matmul(delta2, W[1].T) * (z1 > 0)
    grad_W0 = np.matmul(X.T, delta1)
    dW = [grad_W0 / N + alpha * W[0], grad_W1 / N + alpha * W[1]]
    db = [np.mean(delta1, axis=0)]
    return dW, db

# Hyper-parameters and network initialization
eta = 5e-1
alpha = 1e-6
gamma = 0.99
eps = 1e-3
num_iter = 150
n_H = 256
n = X_train.shape[1]
K = 10

# Initialization
np.random.seed(1127825)
W = [1e-1 * np.random.randn(n, n_H), 1e-1 * np.random.randn(n_H, K)]
b = [np.random.randn(n_H)]

# Gradient Descent: training of the network
gW0 = gW1 = gb0 = 1

for i in range(num_iter):
    dW, db = backprop(W, b, X_train, y_train, alpha)
    gW0 = gamma * gW0 + (1 - gamma) * np.sum(dW[0] ** 2)
    etaW0 = eta / np.sqrt(gW0 + eps)
    W[0] -= etaW0 * dW[0]
    gW1 = gamma * gW1 + (1 - gamma) * np.sum(dW[1] ** 2)
    etaW1 = eta / np.sqrt(gW1 + eps)
    W[1] -= etaW1 * dW[1]
    gb0 = gamma * gb0 + (1 - gamma) * np.sum(db[0] ** 2)
    etab0 = eta / np.sqrt(gb0 + eps)
    b[0] -= etab0 * db[0]
    if i % 30 == 0:
        y_pred = h(X_train, W, b)
        print(f"Iteration {i}: Loss = {loss(y_pred, y_train):.8f}, Accuracy = {np.mean(np.argmax(y_pred, axis=1) == y_train):.4%}")

y_pred_final = h(X_train, W, b)
print(f"Final Loss = {loss(y_pred_final, y_train):.8f}")
print(f"Final Training Accuracy = {np.mean(np.argmax(y_pred_final, axis=1) == y_train):.4%}")
